In [1]:
# Imports 
import openpyxl
import os

In [2]:
os.getcwd()

'c:\\Users\\aa\\Desktop\\Folder\\extract_P&L'

In [3]:
# Define filepaths (duration - 1 min)

excel_name = "Coo_Costs_2016-2023.xlsx"
excel_path = os.path.join(os.getcwd(), excel_name)

workbook = openpyxl.load_workbook(excel_path)

# Select the source sheet and the results sheet
source_sheet = workbook["Gesamtkosten"]         # Excel sheet aus dem die Daten genommen werden
result_sheet = workbook.create_sheet("result")               # Excel sheet in den die Daten einzufügen sind
end_result_sheet = workbook.create_sheet("end result")     

In [5]:
#
# Create the search parameters (duration - 10 sec)
#

# Initialize a set for search parameters in the first column

parameter_column1 = set()                            # first column (= Liste aller Kostenartenbezeichnungen); dazu benötigen wir ein set(), welches werte nur in einfacher Ausfertigung aufnimmt (= keine Mehrfachnennungen innerhalb des sets, sonst würden die Werte doppelt berücksichtigt)
# Iterate over the rows in column B to collect unique search parameters
for cell_b in source_sheet['B'][1:]:
    if cell_b.value is not None:
        parameter_column1.add(cell_b.value)

# Initialize a list for search parameters in the first column               
parameter_column2 = ["2016","2017", "2018", "2019", "2020", "2021", "2022"]         # second column (= Geschäftsjahre)


print("Kostenarten: "+parameter_column1)
print("Jahre: "+parameter_column2)

{'', 'Bez. Inbetrn-leist D', 'Kalk. Lizenzen (POC)', 'Sonst.Beratungskoste', 'Fremdpersonal Fertig', 'Verpflegungspauschal', 'Auftragszinsen (PoC)', 'VV-Zuschlag ALED', 'Bez. Konstr-leist Dr', 'VTGK Intra Group-Mat', 'Eigene Konstr. - Pl', 'Nebenkosten Geldverk', 'Kaufm.Projektleitung', 'Verpackung abw. Meng', 'Lagerentnahmen', 'Ausg.-frachten', 'Porto und Ähnliches', 'VWGK OWN-Lagermat.', 'Eing-u.Zwischenfrach', 'Werbegeschenke-abzf', 'Projektleitung Konst', 'Postdienste vU', 'Eingangszölle', 'Reisekost. n.akt.-fä', 'sonst.bez.Leist. Dri', 'Bewirtungskost-abzf', 'Sonstige Mieten/Pach', 'Bestimmungsmaterial', 'Bez. Inbetrn-leist v', 'Avalprovisionen', 'Reisekost. akt.-fäh.', 'Eigene Konstruktion', 'Kalk. Lizenzen', 'and.vertriebsbed.Auf', 'Reisekost. allg.', 'Wartung/Rep. v. BuGA', 'Eig. Fert. Qualität', 'Eig. Fert. Qualität.', 'Bez. Konstr-leist vU', 'Eig. Fert. AV', 'Monteurauslösungen', 'Kalk. Wagnis OWN-LV', 'MGK OWN-Bestimm.Mat.', 'Transportversicherun', 'Fertigungssonderkost', 'B

In [6]:
# Extract data from the excel sheets (duration - 2 min. )

# Initiate an index for the columns so each year can be stored separate columns

col_index = 1

# initialize variables for the P&L cost centres

material_costs = 0
manufacturing_costs = 0
engineering_project_management_costs = 0
installation_erection_costs = 0
other_costs = 0
overhead_costs = 0

material_cost_parameters = {"Bestimmungsmaterial"}
manufacturing_cost_parameters = {"Konzernzulieferung"}
engineering_project_management_cost_parameters = {"Eigene Konstr. - Pl", "Eigene Konstruktion", "Kaufm.Projektleitung", "Projektleitung Konst" }
installation_erection_cost_parameters = {"Bez. Inbetrn-leist D", "Bez. Inbetrn-leist v", "Bez. Konstr-leist Dr", "Bez. Konstr-leist vU", 
                                         "Bez. Montageleist.Dr", "Eigene Montage / IBN" }
other_cost_parameters = {"and.vertriebsbed.Auf", "Ausg.-frachten", "Avalprovisionen", "Garantieversicherung", "Monteurauslösungen", 
                         "Sonst.Beratungskoste", "sonst.bez.Leist. Dri", "sonstige Dienstleist", "Sonst.Fremdleist.Dri", "Sonstige Vertriebsko", 
                         "Verpackung abw. Meng", "Transportversicherun"
                          }
overhead_cost_parameters = {"MGK Intra Group-Mat" ,"MGK OWN-Bestimm.Mat.", "VTGK Intra Group-Mat", "VTGK OWN-Lagermat.", 
                            "VTGK OWN-LV", "VWGK Intra Group-Mat", "VWGK OWN-Lagermat.", "VWGK OWN-LV"}

cost_parameters = [material_cost_parameters, manufacturing_cost_parameters, engineering_project_management_cost_parameters, installation_erection_cost_parameters, other_cost_parameters, overhead_cost_parameters]


# Iterate over the search parameters in the set for each year contained in the list 
for year in parameter_column2:

    result_sheet.cell(row=1, column=col_index).value = year     # Set the year in row 1
    end_result_sheet.cell(row=1, column=col_index).value = year     # Set the year in row 1

    for i, parameter in enumerate(parameter_column1, start=2):
        sum_parameter = 0
        
        for cell_1, cell_2, cell_3 in zip(source_sheet['B'][1:], source_sheet['E'][1:], source_sheet['X'][1:]): # iterate over 3 columns simultanuously
            if cell_1.value == parameter and cell_2.value == year:
                if cell_3.value is not None:
                    sum_parameter += cell_3.value

        result_sheet.cell(row=i, column=col_index).value = parameter
        result_sheet.cell(row=i, column=col_index + 1).value = sum_parameter

# Calculate cost centres based on parameter matching
        for j, cost_param_list in enumerate(cost_parameters, start=2):

            # If the parameter in the text matches the parameter in our list the costs are allocated to the coresponding cost centre
            if parameter in cost_param_list:
                if j == 1:
                    material_costs += sum_parameter
                elif j == 2:
                    manufacturing_costs += sum_parameter
                elif j == 3: 
                    engineering_project_management_costs += sum_parameter
                elif j == 4:
                    installation_erection_costs += sum_parameter
                elif j == 5:
                    other_costs += sum_parameter
                elif j == 6:
                    overhead_costs += sum_parameter

                print(material_costs)
                print(manufacturing_costs)
                print(engineering_project_management_costs)
                print(installation_erection_costs)
                print(other_costs)
                print(overhead_costs)

# Add the values to the end result sheet
    end_result_sheet.cell(row=2, column=col_index).value = "Material costs"
    end_result_sheet.cell(row=2, column=col_index + 1).value = material_costs

    end_result_sheet.cell(row=3, column=col_index).value = "Manufacturing costs"
    end_result_sheet.cell(row=3, column=col_index + 1).value = manufacturing_costs

    end_result_sheet.cell(row=4, column=col_index).value = "Engineering and Project Management costs"
    end_result_sheet.cell(row=4, column=col_index + 1).value = engineering_project_management_costs

    end_result_sheet.cell(row=5, column=col_index).value = "Installation and Erection costs"
    end_result_sheet.cell(row=5, column=col_index + 1).value = installation_erection_costs

    end_result_sheet.cell(row=6, column=col_index).value = "Other costs"
    end_result_sheet.cell(row=6, column=col_index + 1).value = other_costs

    end_result_sheet.cell(row=7, column=col_index).value = "Overhead costs"
    end_result_sheet.cell(row=7, column=col_index + 1).value = overhead_costs

# Increase the index so the next year can be added separately
    col_index += 2

output_filename = "output.xlsx"
output_path = os.path.join(os.getcwd(), output_filename)

workbook.save(output_path)

0
0
0
0
4260.94
0
0
0
0
0
4260.94
3000
0
0
0
0
26905.819999999996
3000
0
0
0
0
26905.819999999996
3000
0
0
0
0
26905.819999999996
3000
0
0
0
10387
26905.819999999996
3000
0
0
0
10387
26905.819999999996
3810
0
0
0
10387
26905.819999999996
4385.12
0
0
0
10387
26905.819999999996
4385.12
0
0
0
94819
26905.819999999996
4385.12
0
0
0
94819
26905.819999999996
30794.769999999997
0
132613.10999999996
0
94819
26905.819999999996
30794.769999999997
0
132613.10999999996
0
94819
26905.819999999996
30794.769999999997
0
132613.10999999996
0
94819
26905.819999999996
31744.769999999997
0
132613.10999999996
0
212848.35
26905.819999999996
31744.769999999997
0
132613.10999999996
0
212848.35
26905.819999999996
31744.769999999997
0
132613.10999999996
0
212848.35
26905.819999999996
31744.769999999997
0
132613.10999999996
0
212848.35
26905.819999999996
31744.769999999997
0
132613.10999999996
0
212848.35
26905.819999999996
31744.769999999997
0
132613.10999999996
0
212848.35
26905.819999999996
32234.769999999997

In [7]:
print(material_costs)
print(manufacturing_costs)
print(engineering_project_management_costs)
print(installation_erection_costs)
print(other_costs)
print(overhead_costs)

0
1061128.3499999994
433864.74
1306894.1199999996
1535827.7799999998
379806.64
